In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Dataset Parameters
image_height, image_width = 32, 32
batch_size = 20

# Load Dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/MyDrive/Data/train",
    image_size=(image_height, image_width),
    batch_size=batch_size
)
valid_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/MyDrive/Data/validation",
    image_size=(image_height, image_width),
    batch_size=batch_size
)
test_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/MyDrive/Data/test",
    image_size=(image_height, image_width),
    batch_size=batch_size
)

# Class Names
class_names = train_ds.class_names

# Normalize Data
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
valid_ds = valid_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# Model Architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

# Compile Model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train Model
history = model.fit(train_ds, validation_data=valid_ds, epochs=10)

# Evaluate Model
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save TFLite Model
with open("sugarcanemodel.tflite", "wb") as f:
    f.write(tflite_model)
print("TFLite model saved as sugarcanemodel.tflite")
